<a href="https://colab.research.google.com/github/Lord-Kanzler/DS-Unit-4-Sprint-2-Neural-Networks/blob/master/module4-Hyperparameter-Tuning/LS_DS_424_Hyperparameter_Tuning_Assignment_ALEX_KAISER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [0]:
##### Your Code Here #####

In [0]:
import pandas as pd

from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam


In [4]:
# Load the data

URL = 'https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv'
df = pd.read_csv(URL)

print(df.shape)
df.head()

(7043, 21)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [5]:
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [6]:
# data cleaning
df = df.drop(columns='customerID')
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'],errors='coerce')
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [0]:
encode = df.describe(exclude='number').columns.tolist()
encoder = OrdinalEncoder()
encoded = encoder.fit_transform(df[encode])

In [8]:
df_ordinal = pd.DataFrame(data=encoded, columns=encode)
df_ordinal

,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Churn
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0
3,1.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,2.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,1.0,1.0,1.0,1.0,2.0,0.0,2.0,0.0,2.0,2.0,2.0,2.0,1.0,1.0,3.0,0.0
7039,0.0,1.0,1.0,1.0,2.0,1.0,0.0,2.0,2.0,0.0,2.0,2.0,1.0,1.0,1.0,0.0
7040,0.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0
7041,1.0,1.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0


In [9]:
df = df.drop(columns=encode)
df.head()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
0,0,1,29.85,29.85
1,0,34,56.95,1889.50
2,0,2,53.85,108.15
3,0,45,42.30,1840.75
4,0,2,70.70,151.65


In [10]:
df = pd.concat([df, df_ordinal], axis=1)
df.head()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Churn
0,0,1,29.85,29.85,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0
1,0,34,56.95,1889.50,1.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0
2,0,2,53.85,108.15,1.0,0.0,0.0,1.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0
3,0,45,42.30,1840.75,1.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,2.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0,2,70.70,151.65,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0


In [11]:
df.isnull().sum()

SeniorCitizen        0
tenure               0
MonthlyCharges       0
TotalCharges        11
gender               0
Partner              0
Dependents           0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
Churn                0
dtype: int64

In [12]:
df = df.fillna(df['TotalCharges'].mean())
df.isnull().sum()

SeniorCitizen       0
tenure              0
MonthlyCharges      0
TotalCharges        0
gender              0
Partner             0
Dependents          0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
Churn               0
dtype: int64

In [13]:
# train/test splits
train, test = train_test_split(df, test_size=0.3, random_state=42)
train.shape, test.shape

((4930, 20), (2113, 20))

In [0]:
# target = churn
target = 'Churn'
features = df.columns[:-1].tolist()

X_train = train[features]
y_train = train[target]

X_test = test[features]
y_test = test[target]

In [0]:
# scaling data/normalization
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [16]:
# baseline Keras MLP model

model = Sequential([
    Dense(10, input_shape=(19,), activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(
    x = X_train_scaled,
    y = y_train,
    validation_data=(X_test_scaled, y_test),
    epochs = 20
)

Epoch 1/20
155/155 [==============================] - 1s 6ms/step - loss: 0.5918 - accuracy: 0.6744 - val_loss: 0.4842 - val_accuracy: 0.7799
Epoch 2/20
155/155 [==============================] - 1s 5ms/step - loss: 0.4707 - accuracy: 0.7779 - val_loss: 0.4359 - val_accuracy: 0.7946
Epoch 3/20
155/155 [==============================] - 1s 4ms/step - loss: 0.4464 - accuracy: 0.7862 - val_loss: 0.4252 - val_accuracy: 0.7956
Epoch 4/20
155/155 [==============================] - 1s 4ms/step - loss: 0.4389 - accuracy: 0.7888 - val_loss: 0.4206 - val_accuracy: 0.7993
Epoch 5/20
155/155 [==============================] - 1s 5ms/step - loss: 0.4345 - accuracy: 0.7884 - val_loss: 0.4180 - val_accuracy: 0.7989
Epoch 6/20
155/155 [==============================] - 1s 5ms/step - loss: 0.4316 - accuracy: 0.7890 - val_loss: 0.4163 - val_accuracy: 0.8031
Epoch 7/20
155/155 [==============================] - 1s 5ms/step - loss: 0.4293 - accuracy: 0.7919 - val_loss: 0.4150 - val_accuracy: 0.8036
Epoch 

In [22]:
# Hyperperameter tuning using Grid Search
# batch_size

# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(10, input_shape=(19,), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
param_grid = {'batch_size': [4,8,16,32,64,128],
              'epochs': [20]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train_scaled, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7961460471153259 using {'batch_size': 64, 'epochs': 20}
Means: 0.7892494916915893, Stdev: 0.014088242178001819 with: {'batch_size': 4, 'epochs': 20}
Means: 0.7933062791824341, Stdev: 0.010930772891979421 with: {'batch_size': 8, 'epochs': 20}
Means: 0.7882352948188782, Stdev: 0.014102842457291705 with: {'batch_size': 16, 'epochs': 20}
Means: 0.7904665350914002, Stdev: 0.012214254030727227 with: {'batch_size': 32, 'epochs': 20}
Means: 0.7961460471153259, Stdev: 0.012892698404431416 with: {'batch_size': 64, 'epochs': 20}
Means: 0.7904665350914002, Stdev: 0.01329489814883219 with: {'batch_size': 128, 'epochs': 20}


In [23]:
# Hyperperameter tuning using Grid Search
# optimizer

# Function to create model, required for KerasClassifier
def create_model(opt='adam'):
    # create model
    model = Sequential()
    model.add(Dense(10, input_shape=(19,), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
param_grid = {'batch_size': [64],
              'epochs': [20],
              'opt': ['sgd', 'adam', 'nadam', 'rmsprop']}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train_scaled, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7953346967697144 using {'batch_size': 64, 'epochs': 20, 'opt': 'adam'}
Means: 0.781135904788971, Stdev: 0.012706257411119932 with: {'batch_size': 64, 'epochs': 20, 'opt': 'sgd'}
Means: 0.7953346967697144, Stdev: 0.008780869810643676 with: {'batch_size': 64, 'epochs': 20, 'opt': 'adam'}
Means: 0.7892495036125183, Stdev: 0.0078664262864817 with: {'batch_size': 64, 'epochs': 20, 'opt': 'nadam'}
Means: 0.7900608539581299, Stdev: 0.011827490886442879 with: {'batch_size': 64, 'epochs': 20, 'opt': 'rmsprop'}


In [24]:
# Hyperperameter tuning using Grid Search
# learning_rate

# Function to create model, required for KerasClassifier
def create_model(lr=0.1):
    # create model
    opt = Adam(learning_rate=lr)
    model = Sequential()
    model.add(Dense(10, input_shape=(19,), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
param_grid = {'batch_size': [64],
              'lr': [0.001, 0.01, 0.1, 0.2],
              'epochs': [20]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train_scaled, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")


Best: 0.7949290037155151 using {'batch_size': 64, 'epochs': 20, 'lr': 0.1}
Means: 0.7900608539581299, Stdev: 0.012454420559311305 with: {'batch_size': 64, 'epochs': 20, 'lr': 0.001}
Means: 0.7924949407577515, Stdev: 0.010492918178229043 with: {'batch_size': 64, 'epochs': 20, 'lr': 0.01}
Means: 0.7949290037155151, Stdev: 0.011606264010243601 with: {'batch_size': 64, 'epochs': 20, 'lr': 0.1}
Means: 0.7693711876869201, Stdev: 0.03151921496315777 with: {'batch_size': 64, 'epochs': 20, 'lr': 0.2}


In [25]:
# Hyperperameter tuning using Grid Search
# activation_function

# Function to create model, required for KerasClassifier
def create_model(act='relu'):
    # create model
    opt = Adam(learning_rate=.01)
    model = Sequential()
    model.add(Dense(10, input_shape=(19,), activation=act))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
param_grid = {'batch_size': [64],
              'epochs': [20],
              'act': ['sigmoid', 'elu', 'relu', 'selu', 'softplus', 'tanh']}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train_scaled, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.8008113622665405 using {'act': 'selu', 'batch_size': 64, 'epochs': 20}
Means: 0.7914807200431824, Stdev: 0.010254938564536242 with: {'act': 'sigmoid', 'batch_size': 64, 'epochs': 20}
Means: 0.7922920823097229, Stdev: 0.01120952604659438 with: {'act': 'elu', 'batch_size': 64, 'epochs': 20}
Means: 0.7943204998970032, Stdev: 0.013612953549644506 with: {'act': 'relu', 'batch_size': 64, 'epochs': 20}
Means: 0.8008113622665405, Stdev: 0.012677075175169118 with: {'act': 'selu', 'batch_size': 64, 'epochs': 20}
Means: 0.7967545628547669, Stdev: 0.014105755659891726 with: {'act': 'softplus', 'batch_size': 64, 'epochs': 20}
Means: 0.7864097356796265, Stdev: 0.013600853450091923 with: {'act': 'tanh', 'batch_size': 64, 'epochs': 20}


In [28]:
# Hyperperameter tuning using Grid Search
# epochs

# Function to create model, required for KerasClassifier
def create_model(act='relu'):
    # create model
    opt = Adam(learning_rate=.01)
    model = Sequential()
    model.add(Dense(10, input_shape=(19,), activation=act))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
param_grid = {'batch_size': [64],
              'epochs': [20, 40, 60, 80, 100],
              'act': ['selu']}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train_scaled, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7967545628547669 using {'act': 'selu', 'batch_size': 64, 'epochs': 40}
Means: 0.7949290037155151, Stdev: 0.01349150830264191 with: {'act': 'selu', 'batch_size': 64, 'epochs': 20}
Means: 0.7967545628547669, Stdev: 0.009740531414794472 with: {'act': 'selu', 'batch_size': 64, 'epochs': 40}
Means: 0.7963488817214965, Stdev: 0.012444510118486584 with: {'act': 'selu', 'batch_size': 64, 'epochs': 60}
Means: 0.7896551728248596, Stdev: 0.011520867577231458 with: {'act': 'selu', 'batch_size': 64, 'epochs': 80}
Means: 0.7858012080192566, Stdev: 0.012244539746355487 with: {'act': 'selu', 'batch_size': 64, 'epochs': 100}


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?